## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import logging
import os

import pandas as pd
from pyarrow import parquet
import s3fs

import helpers.dbg as dbg
import helpers.env as env
import helpers.printing as prnt

In [2]:
prnt.config_notebook()

# dbg.init_logger(verbosity=logging.DEBUG)
dbg.init_logger(verbosity=logging.INFO)
# dbg.test_logger()
_LOG = logging.getLogger(__name__)

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-26b9c9dd-34a1-43ca-8ac4-d0545f037ca0.json'


In [ ]:
# def print_columns(df: pd.DataFrame) -> None:
#     print("# Columns")
#     print("num_cols=%s" % len(df.columns))
#     print(", ".join(df.columns.tolist()))

## Real-time node

In [3]:
import time

## Test real-time node

In [ ]:
nid = "rtds"
start_date = pd.Timestamp("2010-01-04 09:30:00")
end_date = pd.Timestamp("2010-01-10 09:30:00")

columns = ["close", "volume"]
rtds = dartu.RealTimeDataSource("rtds", columns start_date, end_date)

now = pd.Timestamp("2010-01-04 09:35:00")
rtds.set_now_time(now)
    
rtds.fit()

## Build pipeline

In [ ]:
import dataflow_amp.real_time.utils as dartu
import dataflow_amp.returns.pipeline as darp
import core.dataflow as cdataf
import core.config as cconfig

dag_builder = darp.ReturnsPipeline()
config = dag_builder.get_config_template()

# # Add the source node.
# source_config = cconfig.get_config_from_nested_dict(
#     {
#         "func": cldns.load_single_instrument_data,
#         "func_kwargs": {
#             "start_date": datetime.date(2010, 6, 29),
#             "end_date": datetime.date(2010, 7, 13),
#         },
#     }
# )
# config["load_prices"] = source_config
# config["resample_prices_to_1min", "func_kwargs", "volume_cols"] = ["volume"]
# config["compute_vwap", "func_kwargs", "rule"] = "15T"
# config["compute_vwap", "func_kwargs", "volume_col"] = "volume"

if False:
    from im.kibot.data.config import S3_PREFIX

    ticker = "AAPL"
    file_path = os.path.join(S3_PREFIX, "pq/sp_500_1min", ticker + ".pq")
    source_node_kwargs = {
        "func": cdataf.load_data_from_disk,
        "func_kwargs": {
            "file_path": file_path,
            "start_date": pd.to_datetime("2010-01-04 9:30:00"),
            "end_date": pd.to_datetime("2010-01-04 16:05:00"),
        },
    }
    config["load_prices"] = cconfig.get_config_from_nested_dict(
        source_node_kwargs
    )
    
else:
    start_date = pd.Timestamp("2010-01-04 09:30:00")
    end_date = pd.Timestamp("2010-01-05 09:30:00")
    
    source_node_kwargs = {
        "columns": ["close", "vol"],
        "start_date": start_date,
        "end_date": end_date,
    }
    config["load_prices"] = cconfig.get_config_from_nested_dict({
        "source_node_name": "real_time_synthetic",
        "source_node_kwargs": source_node_kwargs
    })

print(config)

In [26]:
dag = dag_builder.get_dag(config)

In [22]:
if False:
    #nid = "compute_ret_0"
    nid = "load_prices"
    node = dag.get_node("load_prices")
    node.set_now_time(pd.to_datetime("2010-01-06 9:30:00"))

    dict_ = dag.run_leq_node(nid, "fit")

    print(dict_)

run_leq_node:   0%|          | 0/1 [00:00<?, ?it/s]

{'df_out':                          close       vol
2010-01-04 09:30:00   0.265995  0.222227
2010-01-04 09:31:00  -0.040810  0.715719
2010-01-04 09:32:00  -0.006197  0.777261
2010-01-04 09:33:00   0.039911  0.830354
2010-01-04 09:34:00   0.428358  0.526327
...                        ...       ...
2010-01-06 09:26:00 -32.152131 -1.976049
2010-01-06 09:27:00 -31.821100 -1.994653
2010-01-06 09:28:00 -31.869451 -1.814798
2010-01-06 09:29:00 -32.288384 -1.908961
2010-01-06 09:30:00 -31.911481 -2.070091

[2881 rows x 2 columns]}


In [ ]:
# import helpers.printing as hprint
# df = cldns.load_db_example_data()
# print("end_time=[%s, %s]" % (min(df["end_time"]), max(df["end_time"])))
# print(df.shape)


# datetime_ = pd.Timestamp("2021-07-22 20:01:00-00:00")
# print(datetime_)
# df = cldns.get_db_data(datetime_)
# print("end_time=[%s, %s]" % (min(df["end_time"]), max(df["end_time"])))
# print(df.shape)

# df.head()

In [ ]:
#df[["start_time", "end_time", "timestamp_db"]]

In [ ]:
node = dag.get_node("load_prices")
node.reset_now_time()
    
for now in dartu.get_now_time(start_date, end_date):
    print("now=", now)
    execute = dartu.is_dag_to_execute(now)
    if execute:
        print("Time to execute the DAG")
        node = dag.get_node("load_prices")
        node.set_now_time(now)
        #
        sink = dag.get_unique_sink()
        dict_ = dag.run_leq_node(sink, "fit")
        print(dict_["df_out"].tail(3))

now= 2010-01-04 09:30:00
Time to execute the DAG


run_leq_node:   0%|          | 0/6 [00:00<?, ?it/s]

                     vwap  twap  last bar_start_timestamp    minute  twap_ret_0  vwap_ret_0
2010-01-04 09:30:00   NaN   NaN   NaN 2010-01-04 09:25:00  21043290         NaN         NaN
now= 2010-01-04 09:31:00
now= 2010-01-04 09:32:00
now= 2010-01-04 09:33:00
now= 2010-01-04 09:34:00
now= 2010-01-04 09:35:00
Time to execute the DAG


run_leq_node:   0%|          | 0/6 [00:00<?, ?it/s]

                         vwap     twap      last bar_start_timestamp    minute  twap_ret_0  vwap_ret_0
2010-01-04 09:30:00       NaN      NaN       NaN 2010-01-04 09:25:00  21043290         NaN         NaN
2010-01-04 09:35:00  0.223739  0.23321  0.153984 2010-01-04 09:30:00  21043295         NaN         NaN
now= 2010-01-04 09:36:00
now= 2010-01-04 09:37:00
now= 2010-01-04 09:38:00
now= 2010-01-04 09:39:00
now= 2010-01-04 09:40:00
Time to execute the DAG


...

run_leq_node:   0%|          | 0/6 [00:00<?, ?it/s]

                     vwap  twap  last bar_start_timestamp    minute  twap_ret_0  vwap_ret_0
2010-01-05 20:40:00   NaN   NaN   NaN 2010-01-05 20:35:00  21045400         NaN         NaN
2010-01-05 20:45:00   NaN   NaN   NaN 2010-01-05 20:40:00  21045405         NaN         NaN
2010-01-05 20:50:00   NaN   NaN   NaN 2010-01-05 20:45:00  21045410         NaN         NaN
now= 2010-01-05 20:51:00
now= 2010-01-05 20:52:00
now= 2010-01-05 20:53:00
now= 2010-01-05 20:54:00
now= 2010-01-05 20:55:00
Time to execute the DAG


run_leq_node:   0%|          | 0/6 [00:00<?, ?it/s]

                     vwap  twap  last bar_start_timestamp    minute  twap_ret_0  vwap_ret_0
2010-01-05 20:45:00   NaN   NaN   NaN 2010-01-05 20:40:00  21045405         NaN         NaN
2010-01-05 20:50:00   NaN   NaN   NaN 2010-01-05 20:45:00  21045410         NaN         NaN
2010-01-05 20:55:00   NaN   NaN   NaN 2010-01-05 20:50:00  21045415         NaN         NaN
now= 2010-01-05 20:56:00
now= 2010-01-05 20:57:00
now= 2010-01-05 20:58:00
now= 2010-01-05 20:59:00
now= 2010-01-05 21:00:00
Time to execute the DAG


run_leq_node:   0%|          | 0/6 [00:00<?, ?it/s]